In [1]:
from langchain_community.vectorstores import Qdrant
from langchain_core.documents import Document
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from typing import List, Dict
from langchain.chains import RetrievalQA
import os

def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder and its subfolders
    for root, _, files in os.walk(data_path):
        for filename in files:
            if filename.endswith('.txt'):
                file_path = os.path.join(root, filename)
                loader = TextLoader(file_path, encoding='utf-8')
                docs.extend(loader.load())

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )

    chunk_size = 512
    chunk_overlap = 0

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    chunked_docs = []

    for doc in docs:
        md_header_splits = markdown_splitter.split_text(doc.page_content)
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs

In [2]:
data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/data'
chunked_data = load_and_chunk_data(data_path)

In [3]:
from langchain import PromptTemplate, FewShotPromptTemplate


examples = [{
    "context": "### B. Tổ hợp xét tuyển :A00 (Toán, Vật lý, Hóa học) và A01 (Toán, Vật lý, Tiếng Anh).Điểm trúng tuyển áp dụng cho A00 và A01 như nhau, điểm xét theo phía Bắc, phía Nam và giới tính Nam, Nữ.",
    "analysis": '''
            Tiêu chí xét tuyển có phân biệt theo khu vực địa lý và giới tính không?,
            Điểm trúng tuyển có sự khác biệt giữa các tổ hợp xét tuyển A00 và A01 không?,
            Những tổ hợp môn nào được sử dụng để xét tuyển vào trường?
'''
}, 
{
    "context": "Thí sinh có thường trú phía Bắc: Thí sinh Nam có điểm trúng tuyển năm 2021 là 25,90 điểm, năm 2022 là 26,05 điểm, và năm 2023 là 24,91 điểm. Thí sinh Nữ có điểm trúng tuyển năm 2021 là 28,05 điểm, năm 2022 là 28,15 điểm, và năm 2023 là 26,87 điểm.",
    "analysis": '''
            Điểm trúng tuyển năm 2022 của thí sinh Nam có thường trú phía Bắc là bao nhiêu?,
            Thí sinh Nữ có thường trú phía Bắc có điểm trúng tuyển cao nhất vào năm nào?,
            So sánh điểm trúng tuyển năm 2021 giữa thí sinh Nam và thí sinh Nữ có thường trú phía Bắc, ai có điểm cao hơn?
'''
}

]


# Định nghĩa mẫu cho mỗi ví dụ
example_template = """
Examples :\n
<Input>
context: {context}
</Input>\n\n
<Output>
Analysis: {analysis}
</Output>
"""

example_prompt = PromptTemplate(
    input_variables=["context"],
    template=example_template
)

# Định nghĩa prompt chính
prefix = "Dựa vào context sau để tạo ra 3 câu hỏi. Cấu hỏi tạo ra phải có ý nghĩa và có thể trả lời được. Cấu hỏi bằng tiếng việt và có định dạng như sau:"
suffix = """
context: {context}
analysis:
"""

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["context"],
    example_separator="\n\n"
)

In [4]:
print(example_prompt.format(**examples[0]))


Examples :

<Input>
context: ### B. Tổ hợp xét tuyển :A00 (Toán, Vật lý, Hóa học) và A01 (Toán, Vật lý, Tiếng Anh).Điểm trúng tuyển áp dụng cho A00 và A01 như nhau, điểm xét theo phía Bắc, phía Nam và giới tính Nam, Nữ.
</Input>


<Output>
Analysis: 
            Tiêu chí xét tuyển có phân biệt theo khu vực địa lý và giới tính không?,
            Điểm trúng tuyển có sự khác biệt giữa các tổ hợp xét tuyển A00 và A01 không?,
            Những tổ hợp môn nào được sử dụng để xét tuyển vào trường?

</Output>



In [18]:
new_example =   {
        "context": "Học viện Kỹ thuật quân sự đang đào tạo 16 ngành, 50 chuyên ngành trong các lĩnh vực: Công nghệ thông tin, An toàn thông tin, Khoa học máy tính, Kỹ thuật điều khiển và tự động hóa, Kỹ thuật tên lửa, Vũ khí, Kỹ thuật cơ khí, Kỹ thuật cơ khí động lực, Kỹ thuật xây dựng, Kỹ thuật trắc địa - bản đồ, v.v.Học viện đào tạo kỹ sư chất lượng cao với 02 chuyên ngành: An ninh hệ thống thông tin, Thông tin."
    }

In [19]:
prompt = few_shot_prompt_template.format(
    context=new_example["context"]
)
# ai_msg = llm.invoke(prompt)
# ai_msg

print(prompt.format(**examples[0]))


Dựa vào context sau để tạo ra 3 câu hỏi. Cấu hỏi tạo ra phải có ý nghĩa và có thể trả lời được. Cấu hỏi bằng tiếng việt và có định dạng như sau:


Examples :

<Input>
context: ### B. Tổ hợp xét tuyển :A00 (Toán, Vật lý, Hóa học) và A01 (Toán, Vật lý, Tiếng Anh).Điểm trúng tuyển áp dụng cho A00 và A01 như nhau, điểm xét theo phía Bắc, phía Nam và giới tính Nam, Nữ.
</Input>


<Output>
Analysis: 
            Tiêu chí xét tuyển có phân biệt theo khu vực địa lý và giới tính không?,
            Điểm trúng tuyển có sự khác biệt giữa các tổ hợp xét tuyển A00 và A01 không?,
            Những tổ hợp môn nào được sử dụng để xét tuyển vào trường?

</Output>



Examples :

<Input>
context: Thí sinh có thường trú phía Bắc: Thí sinh Nam có điểm trúng tuyển năm 2021 là 25,90 điểm, năm 2022 là 26,05 điểm, và năm 2023 là 24,91 điểm. Thí sinh Nữ có điểm trúng tuyển năm 2021 là 28,05 điểm, năm 2022 là 28,15 điểm, và năm 2023 là 26,87 điểm.
</Input>


<Output>
Analysis: 
            Điểm trúng tuyển năm 2

In [10]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
 ## Replace to real LLMs (Cohere / Groq / OpenAI)

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


In [11]:
def extract_questions_and_save_to_json(input_str, output_filename):
    # Split the input string to extract questions
    questions = input_str.split("Analysis:")[1].strip().split(",\n")
    questions = [question.strip() for question in questions if question.strip()]
    with open(output_filename, 'a', encoding='utf-8') as file:
    # Save the questions into a JSON file
        for q in questions:
            file.write(q+'\n')

In [30]:
import random 

def update_llm_api_key():
    random_number = random.randint(1, 10)
    key = f'llm_api_{random_number}'
    return key

'llm_api_10'

In [12]:
from tqdm import tqdm
for item in tqdm(chunked_data, desc="Gen data"):
    llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1,api_key= os.getenv(update_llm_api_key()))
    prompt = few_shot_prompt_template.format(
    context=item.page_content)
    output=llm.invoke(prompt)
    extract_questions_and_save_to_json(output.content, 'questions.txt')


Gen data:   3%|▎         | 15/582 [00:30<19:10,  2.03s/it]


KeyboardInterrupt: 